In [ ]:
# Requirements for this notebook to run: pandas, geopandas, shapely, pyproj
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json

import pandas
from shapely.geometry import shape

from eocalc.context import Pollutant
from eocalc.methods.base import DateRange
from eocalc.methods.naive import TropomiMonthlyMeanAggregator

In [ ]:
# Select region
region_name = "Saxony"
with open("data/regions/roughly_saxonia.geo.json", 'r') as geojson_file:
    region = shape(json.load(geojson_file)["geometry"])

# Select dates
periods = []
start = TropomiMonthlyMeanAggregator.earliest_start_date()
if start.day != 1:
    start = (start + relativedelta(months=1)).replace(day=1)

while start < TropomiMonthlyMeanAggregator.latest_end_date():
    periods += [DateRange(start, start + relativedelta(months=1) - relativedelta(days=1))]
    start += relativedelta(months=1)

# Select pollutant
pollutant = Pollutant.NO2

In [ ]:
results = {} # results will be put here as results[<period index>][<data>]
start = datetime.now()

for index, period in enumerate(periods):
    results[index] = TropomiMonthlyMeanAggregator().run(region, period, pollutant)
    print(f"Done with period '{period}'")

print(f"All finished in {datetime.now()-start}.")

In [ ]:
emissions_over_time = pandas.DataFrame({f"{pollutant.name} emissions [kt]": 42.0}, index=[f"{period.start:%Y-%m}" for period in periods])

for index, result in results.items():
    table = result[TropomiMonthlyMeanAggregator.TOTAL_EMISSIONS_KEY]
    emissions_over_time[f"{pollutant.name} emissions [kt]"][f"{periods[index].start:%Y-%m}"] = table.iloc[-1, 0]

emissions_over_time.plot.line(figsize=(20, 10), rot=90)

In [ ]:
# Optional: show monthly emissions as animation
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import contextily as ctx

fig, ax = plt.subplots(figsize=(10, 10))

def animate(frame_number):
    ax.clear()
    ax.set_title(f"{periods[frame_number].start:%Y-%m}")
    data = results[frame_number][TropomiMonthlyMeanAggregator.GRIDDED_EMISSIONS_KEY]
    ctx.add_basemap(data.to_crs(epsg=3857).plot(f"Total {Pollutant.NO2.name} emissions [kg]", ax=ax, alpha=0.7, edgecolor='k'))

slideshow = animation.FuncAnimation(fig, animate, frames=len(periods), interval=300, repeat=True, blit=False)
plt.close()  # Prevent extra plot output below animation
# slideshow.save("emissions.gif", writer=animation.PillowWriter(fps=1))

HTML(slideshow.to_jshtml())